In [1]:
import pandas as pd
from src.firefall import FirefallService
from src.imss import IMSService
from src.utils import Utils

adobe_env = 'prod'
ims_service = IMSService(adobe_env)
firefall_service = FirefallService(ims_service.ACCESS_TOKEN, 'prod')

utils = Utils()

#One time run to calculate and save embeddings
#document_dir = "data/lovesac/scraped_content"
#collection_name = 'lovesac'
#utils.calculate_save_embeddings(document_dir, collection_name)

#Read the broken backlinks and sitemap URLs from the files
with open('data/lovesac/broken_urls.txt') as f:
    broken_backlinks = f.read().splitlines()

with open('data/lovesac/sitemap_urls.txt') as f:
    sitemap_urls = f.read().splitlines()

<Response [200]>


In [2]:
broken_backlink = broken_backlinks[1]
df = utils.search_for_similar(broken_backlink, 'lovesac', k=3)  
df.insert(0, 'Broken Back Link', broken_backlink) 
df

,Broken Back Link,File Name,Distance,Index
0,https://www.lovesac.com/footsac-blanket-mousse...,footsac-blanket-dove-boucle.md,5.408681,146
1,https://www.lovesac.com/footsac-blanket-mousse...,room-for-two-footsac-blanket-dove-channeled-ph...,5.166362,149
2,https://www.lovesac.com/footsac-blanket-mousse...,room-for-two-footsac-blanket-midnight-mink-phu...,4.920412,63


In [3]:
prompt = f'''

As an SEO expert tasked with fixing broken backlinks, you will receive an alternative URL from the same website, selected based on similarity indices. 
Please be aware that these suggestions might not be completely accurate.

Carefully review the provided URLs. 
If the suggested URL does not seem to be an exact match, opt to redirect to the homepage or, if it pertains to a product, the relevant category page.

Matching Process:
- Identify the keywords after the last slash in the URLs.
- Analyze the keywords semantically to understand their meaning.
- Evaluate similarity: If the keywords are semantically similar, the URL is considered a match.
Remember, the goal is to redirect users to a page that closely matches their original intent. 
It is preferable to redirect to a related category page or the main page rather than a completely unrelated one.

For example, for the following example, as the URLs were not a good match, and broken link is a product page, then it is better to redirect to the category page. (if it is included in the URL, do not make up a URL)
Broken link: https://www.lovesac.com/sacs/bigone.html
Alternative URL link: https://www.lovesac.com/financing
Suggested URL: https://www.lovesac.com/sacs/

Now, give suggested URL for the following:
Here is the broken link: {df['Broken Back Link'][0]}
Here is the suggested link: {df['Broken Back Link'][0]}

The output should be a JSON containing the suggested URL and AI rationale.

'''
response = firefall_service.completions(11, prompt)
response['generations'][0][0]['message']['content']

200 {"conversation_identifier":null,"query_id":null,"llm_type":"azure_chat_openai","generations":[[{"text":"{\n  \"suggested_url\": \"https://www.lovesac.com/blankets/\",\n  \"ai_rationale\": \"The broken link seems to be a product page for a footsac blanket in mousse phur. The suggested URL is the category page for blankets, which closely matches the original intent of the user.\"\n}","generation_info":{"finish_reason":"stop","content_filter_results":{"hate":{"filtered":false,"severity":"safe"},"self_harm":{"filtered":false,"severity":"safe"},"sexual":{"filtered":false,"severity":"safe"},"violence":{"filtered":false,"severity":"safe"}}},"type":"ChatGeneration","message":{"content":"{\n  \"suggested_url\": \"https://www.lovesac.com/blankets/\",\n  \"ai_rationale\": \"The broken link seems to be a product page for a footsac blanket in mousse phur. The suggested URL is the category page for blankets, which closely matches the original intent of the user.\"\n}","additional_kwargs":{},"typ

'{\n  "suggested_url": "https://www.lovesac.com/blankets/",\n  "ai_rationale": "The broken link seems to be a product page for a footsac blanket in mousse phur. The suggested URL is the category page for blankets, which closely matches the original intent of the user."\n}'